In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Import statsmodels package for training a linear regression model.
import statsmodels.formula.api as sm

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

ImportError: No module named 'sklearn.model_selection'

# Import and Name Columns

In [2]:
#Read in the data from a csv file using Pandas
df = pd.read_csv("bus_data/cleaned_data/line15.csv", low_memory=False, header=None)
df.columns = ["Timestamp", "LineID", "Direction", "JourneyPatternID", "TimeFrame", 
              "VehicleJourneyID", "Operator", "Congestion", "Lon", "Lat", 
              "Delay", "BlockID", "VehicleID", "StopID", "AtStop"]

In [3]:
#Select all columns of type 'object'
object_columns = df.select_dtypes(['object']).columns

In [4]:
#Convert selected columns to type 'category'
for column in object_columns:
    df[column] = df[column].astype('category')   

In [5]:
# Convert Unix timestamp to datetime

# Take uneccesary trailing zeroes off of unix timestamp. 
df['Timestamp'] = df['Timestamp'].apply(lambda x: x//1000000)

# Convert the timestamp
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')

In [6]:
# Add day of week column
df['Day'] = df['Timestamp'].dt.dayofweek

In [7]:
# Add hour of day column
df['Hour'] = df['Timestamp'].dt.hour

In [8]:
# Convert some features to categorical
for column in ['LineID', 'Direction','VehicleJourneyID', 'Congestion', 'BlockID', 'VehicleID', 'AtStop','Day','Hour']:
    df[column] = df[column].astype('category')

In [9]:
df = df.drop('BlockID', 1)
df = df.drop('Operator', 1)
df = df.drop('Delay', 1)
df = df.drop('Congestion', 1)

In [10]:
df = df.loc[df['JourneyPatternID'] == '00150001']

In [11]:
df.dtypes

Timestamp           datetime64[ns]
LineID                    category
Direction                 category
JourneyPatternID          category
TimeFrame                 category
VehicleJourneyID          category
Lon                        float64
Lat                        float64
VehicleID                 category
StopID                    category
AtStop                    category
Day                       category
Hour                      category
dtype: object

In [12]:
df.shape

(607458, 13)

### Continuous Columns

In [97]:
#Drop the duplicates, but keep the first occurance of when they occur
df = df.drop_duplicates(keep='first')

In [98]:
df.shape

(2254310, 13)

In [102]:
df.TimeFrame.nunique()

26

In [13]:
df.TimeFrame.value_counts()

2012-11-07    28406
2012-11-09    28215
2012-11-27    28075
2012-11-23    28039
2012-11-08    28016
2012-11-14    27946
2012-11-06    27805
2012-11-19    27772
2012-11-16    27553
2012-11-29    27500
2012-11-22    27452
2012-11-15    27393
2012-11-20    27342
2012-11-26    27264
2012-11-28    26797
2012-11-13    26564
2012-11-12    26488
2012-11-21    26394
2012-11-30    24179
2012-11-10    19086
2012-11-17    18784
2012-11-24    17595
2012-11-18    11262
2012-11-11    10744
2012-11-25    10673
2012-11-05      114
Name: TimeFrame, dtype: int64

33 columns dropped

In [14]:
for i in df.TimeFrame.unique():
    print(i)

2012-11-05
2012-11-06
2012-11-07
2012-11-08
2012-11-09
2012-11-10
2012-11-11
2012-11-12
2012-11-13
2012-11-14
2012-11-15
2012-11-16
2012-11-17
2012-11-18
2012-11-19
2012-11-20
2012-11-21
2012-11-22
2012-11-23
2012-11-24
2012-11-25
2012-11-26
2012-11-27
2012-11-28
2012-11-29
2012-11-30


In [53]:
# durations = []

# for date in df.TimeFrame.unique():    
#     temp_df = df[(df.VehicleJourneyID == 5826) & (df.TimeFrame == date)]
    
#     duration = temp_df.Timestamp.max()-temp_df.Timestamp.min()

#     durations.append(duration)

In [54]:
# durations = []

# temp_df = df[df.TimeFrame.unique()[0]]

# inner_df = df[temp_dfdf.VehicleJourneyID == temp_df.VehicleJourneyID.unique()[0]]
    
# duration = inner_df.Timestamp.max()-inner_df.Timestamp.min()

# durations.append(duration)

In [55]:
# durations = []

# for date in df.TimeFrame.unique():    
#     temp_df = df[df.TimeFrame == date]
    
#     for journey in temp_df.VehicleJourneyID.unique():
        
#         inner_df = df[df.VehicleJourneyID == journey]
    
#         duration = inner_df.Timestamp.max()-inner_df.Timestamp.min()
#         durations.append(duration)

In [56]:
durations = []
columns = ['hour','day','duration']
rows = []

for date in df.TimeFrame.unique():    
    temp_df = df[df.TimeFrame == date]
    
    for journey in temp_df.VehicleJourneyID.unique():
        
        inner_df = temp_df[temp_df.VehicleJourneyID == journey]
        
        row = {}
    
        duration = inner_df.Timestamp.max()-inner_df.Timestamp.min()
        
        if (duration > datetime.timedelta(minutes=30)):
        
            row['hour'] = inner_df.Timestamp.min().hour
            row['day'] = inner_df.Timestamp.min().dayofweek
            row['duration'] = duration.seconds//60

            rows.append(row)

In [57]:
pp.pprint(rows)

[{'day': 1, 'duration': 105, 'hour': 6},
 {'day': 1, 'duration': 92, 'hour': 6},
 {'day': 1, 'duration': 93, 'hour': 6},
 {'day': 1, 'duration': 94, 'hour': 6},
 {'day': 1, 'duration': 148, 'hour': 6},
 {'day': 1, 'duration': 96, 'hour': 6},
 {'day': 1, 'duration': 108, 'hour': 6},
 {'day': 1, 'duration': 107, 'hour': 7},
 {'day': 1, 'duration': 107, 'hour': 7},
 {'day': 1, 'duration': 103, 'hour': 7},
 {'day': 1, 'duration': 106, 'hour': 7},
 {'day': 1, 'duration': 117, 'hour': 7},
 {'day': 1, 'duration': 112, 'hour': 7},
 {'day': 1, 'duration': 115, 'hour': 8},
 {'day': 1, 'duration': 108, 'hour': 8},
 {'day': 1, 'duration': 110, 'hour': 8},
 {'day': 1, 'duration': 109, 'hour': 8},
 {'day': 1, 'duration': 106, 'hour': 8},
 {'day': 1, 'duration': 104, 'hour': 8},
 {'day': 1, 'duration': 106, 'hour': 9},
 {'day': 1, 'duration': 107, 'hour': 9},
 {'day': 1, 'duration': 103, 'hour': 9},
 {'day': 1, 'duration': 107, 'hour': 9},
 {'day': 1, 'duration': 94, 'hour': 9},
 {'day': 1, 'duration

In [58]:
basetable = pd.DataFrame(rows)

In [64]:
basetable.duration.max()

254

In [40]:
# boi = df.groupby(['VehicleJourneyID','TimeFrame']).size().reset_index().rename(columns={0:'count'})
# boi

,VehicleJourneyID,TimeFrame,count
0,4727,2012-11-19,1
1,4727,2012-11-21,1
2,4727,2012-11-22,1
3,4734,2012-11-20,1
4,4734,2012-11-22,1
5,4734,2012-11-23,1
6,4739,2012-11-23,1
7,4762,2012-11-26,1
8,4762,2012-11-30,1
9,4767,2012-11-30,1
